In [1]:
import torch
from pykeops.torch import LazyTensor
from gpytorch.kernels.keops import RBFKernel
from gpytorch.lazy import LazyEvaluatedKernelTensor

In [2]:
torch.set_default_dtype(torch.float64)

In [3]:
n = 1000
d = 3
sigma = 1.0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

A = torch.randn(n, d, device=device)
B = torch.randn(n, n, device=device)

PyKeops -- doesn't work

In [4]:
# Ai_lazy = LazyTensor(A[:, None, :])
# Aj_lazy = LazyTensor(A[None, :, :])
# # Compute the squared Euclidean distance matrix
# D_lazy = ((Ai_lazy - Aj_lazy) ** 2).sum(dim=2)
# D_lazy - B

GPyTorch

In [5]:
kernel = RBFKernel()
kernel.lengthscale = torch.tensor(sigma)
kernel = kernel.to(device)
# print(kernel.lengthscale)
# print(kernel.lengthscale.device)
K_lazy = kernel(A, A)

In [6]:
# Compute the actual kernel matrix via pytorch
K_true = torch.exp(-((A[:, None, :] - A[None, :, :])
                   ** 2).sum(dim=2)/(2 * sigma ** 2))

In [7]:
# torch.sum(K_lazy ** 2)

In [8]:
print(torch.allclose(K_lazy @ B, K_true @ B))
print(torch.allclose(B @ K_lazy, B @ K_true))

True
True


In [9]:
print(torch.allclose((K_lazy - B) @ B, (K_true - B) @ B))
print(torch.allclose((K_lazy - B).T @ B, (K_true - B).T @ B))
print(torch.allclose(B @ (K_lazy - B), B @ (K_true - B)))

True
True
True


In [10]:
print(torch.allclose(B[:, :100] @ (K_lazy - B)
      [:100], B[:, :100] @ (K_true - B)[:100]))

True


In [11]:
K_lazy.device

device(type='cuda', index=0)

In [12]:
K_lazy.to_dense()

tensor([[1.0000, 0.0506, 0.0489,  ..., 0.0216, 0.7564, 0.1918],
        [0.0506, 1.0000, 0.5184,  ..., 0.2204, 0.2087, 0.0298],
        [0.0489, 0.5184, 1.0000,  ..., 0.8263, 0.2297, 0.0212],
        ...,
        [0.0216, 0.2204, 0.8263,  ..., 1.0000, 0.1174, 0.0061],
        [0.7564, 0.2087, 0.2297,  ..., 0.1174, 1.0000, 0.2013],
        [0.1918, 0.0298, 0.0212,  ..., 0.0061, 0.2013, 1.0000]],
       device='cuda:0', grad_fn=<MatmulBackward>)

In [13]:
(K_lazy - B).to_dense()

tensor([[ 0.8660, -0.6374,  1.4401,  ..., -1.0699,  2.0569, -0.0062],
        [-0.5327, -0.2933,  1.4188,  ..., -0.2032,  0.0468,  0.4302],
        [-0.2271,  0.1803,  1.6148,  ..., -0.1274, -1.7404,  0.2231],
        ...,
        [ 0.0420,  2.5854,  0.3486,  ...,  0.6906,  0.3283, -1.0672],
        [ 0.8660,  0.1091, -0.8672,  ...,  0.2745,  3.3221, -0.1215],
        [ 1.6261,  0.3557, -1.2657,  ..., -0.7316,  0.6237,  0.5947]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [14]:
type(K_lazy-B)

linear_operator.operators.sum_linear_operator.SumLinearOperator

In [15]:
from linear_operator.operators import DenseLinearOperator, CatLinearOperator

In [16]:
B_linop = DenseLinearOperator(B)
B_K_lazy_linop = CatLinearOperator(B_linop, K_lazy, dim=1)

In [17]:
B_K_lazy_linop.shape

torch.Size([1000, 2000])

In [18]:
from scalable_gp_inference.gp_inference_rhs import GPInferenceRHS

In [20]:
rhs = GPInferenceRHS(
    targets=torch.randn(n, 1, device=device),
    X=A,
    kernel_fn=kernel,
    tst_idx = torch.arange(100)
)

In [21]:
rhs.shape

torch.Size([1000, 101])